## Tallies

Any tally in OpenMC can be described with the following form:

$$ 
 X = \underbrace{\int d\mathbf{r} \int d\mathbf{\Omega} \int
    dE}_{\text{filters}} \underbrace{f(\mathbf{r}, \mathbf{\Omega},
    E)}_{\text{scores}} \underbrace{\psi (\mathbf{r}, \mathbf{\Omega}, E)}_{\text{angular flux}}
$$

where filters set the limits of the integrals and the scoring function is convolved with particle information (e.g. reaction type, current material, etc.).

In [ ]:
import openmc
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
pincell_model = openmc.examples.pwr_pin_cell()

In [ ]:
pincell_model

In [ ]:
pincell_model.geometry.root_universe.plot()

In [ ]:
material_dict = pincell_model.geometry.get_all_materials()

In [ ]:
material_dict

In [ ]:
material_dict[1]

In [ ]:
pincell_model.tallies

## Applying Tallies

In [ ]:
fission_tally = openmc.Tally(name='fission tally')

In [ ]:
fission_tally

<div class="alert alert-block alert-info">
    <a href="https://docs.openmc.org/en/stable/usersguide/tallies.html?highlight=tally%20scores#scores">Full list of OpenMC scores and their units</a>
    <br/>
    <a>(Scores can also be specified using ENDF MT reaction numbers)</a>
</div>

In [ ]:
fission_tally.scores = ['fission', 'nu-fission', 'kappa-fission', 'heating-local']
fission_tally

In [ ]:
energy_filter = openmc.EnergyFilter.from_group_structure('CASMO-70')
len(energy_filter.bins)

In [ ]:
spectrum_tally = openmc.Tally(name="spectrum tally")
spectrum_tally.filters = [energy_filter]
spectrum_tally.scores = ['flux']
print(spectrum_tally)

In [ ]:
tallies = openmc.Tallies([fission_tally, spectrum_tally])

In [ ]:
pincell_model.tallies = tallies

In [ ]:
pincell_model.settings.batches = 50
pincell_model.settings.inactive = 10
pincell_model.settings.particles = 5000

In [ ]:
sp_filename = pincell_model.run()

In [ ]:
sp_filename

In [ ]:
!ls

In [ ]:
!head -n 20 tallies.out

In [ ]:
statepoint = openmc.StatePoint(sp_filename)

In [ ]:
statepoint.tallies

In [ ]:
fission_tally

In [ ]:
fission_tally_out = statepoint.get_tally(name='fission tally')

In [ ]:
fission_tally_out

In [ ]:
df = fission_tally_out.get_pandas_dataframe()
df

In [ ]:
fission_tally_out.mean

In [ ]:
fission_tally_out.std_dev

In [ ]:
fission_tally_out.mean.shape

In [ ]:
fission_rate = fission_tally_out.get_values(scores=['fission'])

In [ ]:
fission_rate = fission_rate.flat[0]

In [ ]:
fission_rate

In [ ]:
kappa_fission = fission_tally_out.get_values(scores=['kappa-fission']).flat[0]

In [ ]:
kappa_fission

In [ ]:
heating_local = fission_tally_out.get_values(scores=['heating-local']).flat[0]

In [ ]:
heating_local

In [ ]:
nu_fission = fission_tally_out.get_values(scores=['nu-fission']).flat[0]

In [ ]:
# fission rate - rate / source-particle 
# energy dep. - eV / source-particle
ev_per_fission = kappa_fission / fission_rate
mev_per_fission = ev_per_fission * 1e-6
print("MeV per fission: {:.3f}".format(mev_per_fission))

In [ ]:
ev_per_fission = heating_local / fission_rate
mev_per_fission = ev_per_fission * 1e-6
print("MeV per fission: {:.3f}".format(mev_per_fission))

<div class="alert alert-block alert-info">
<b>A quick aside on how statepoint objects interact with summary files:</b>


The `openmc.statepoint` object will read information from the `summary.h5` file if one is present, keeping that file open in the Python interpreter. The open `summary.h5` file can interfere with the initialization of subsequent OpenMC simulations. It is recommended that information be extracted from statepoints within a [context manager](https://book.pythontips.com/en/latest/context_managers.html) as we do here. Alternatively, making sure to call the `openmc.StatePoint.close` method will work also. For more details please look to the [relevant section in the user's guide](https://docs.openmc.org/en/stable/usersguide/troubleshoot.html#runtimeerror-failed-to-open-hdf5-file-with-mode-w-summary-h5).   
</div>



In [ ]:
statepoint.close()

In [ ]:
# equivalent to `statepoint = openmc.StatePoint(sp_filename)`
with openmc.StatePoint(sp_filename) as statepoint:
    # run some code
    print(statepoint.k_combined)

## Normalizing Tallies

The combination of the following tally values and power provide us with the source normalization needed as follows:


$$ \text{neutron source} [\frac{n}{s}] = \text{power} [\frac{J}{s}] \times \frac{1}{1.6\times 10^{-19}} [\frac{eV}{J}] \times \frac{1}{\text{heat per fission} [\frac{eV}{fission}]} \times \text{neutrons per fission} [\frac{n}{fission}]$$ 

In [ ]:
pincell_power = 60.0e3 / 300 # in Watts / cm

heat_per_fission = heating_local / fission_rate
J_to_eV = 1 / 1.6e-19

neutron_source = pincell_power * J_to_eV * (1 / heat_per_fission) * nu_fission
print(f'Neutron source: {neutron_source:.2e} n/s')

## Plotting the Flux Spectrum

In [ ]:
with openmc.StatePoint(sp_filename) as statepoint:
    spectrum_tally_out = statepoint.get_tally(id=spectrum_tally.id)

In [ ]:
plt.figure(figsize=(16, 9))

bin_boundaries = np.unique(energy_filter.bins).copy()

print(np.min(bin_boundaries))
print(np.max(bin_boundaries))

In [ ]:
bin_boundaries[0] = 1e-10

flux = spectrum_tally_out.mean.flatten() # particle / source
flux *= neutron_source

# need to normalize flux by the log-width of the energy bins
log_de = np.log10(bin_boundaries[1:]/bin_boundaries[:-1])
flux = flux / log_de


plt.step(np.unique(energy_filter.bins)[:-1], flux)
plt.xscale('log')
plt.show()

## Reaction Rates by Material

In [ ]:
material_filter = openmc.MaterialFilter(pincell_model.materials)

In [ ]:
material_tally = openmc.Tally()
material_tally.filters = [material_filter]
material_tally.scores = ['absorption', 'scatter', 'fission']
pincell_model.tallies = [material_tally]

In [ ]:
sp_filename = pincell_model.run()

In [ ]:
with openmc.StatePoint(sp_filename) as sp:
    tally = list(sp.tallies.values())[0]
    absorption = tally.get_values(scores=['absorption']).flatten()
    scatter = tally.get_values(scores=['scatter']).flatten()
    fission = tally.get_values(scores=['fission']).flatten()
    df = tally.get_pandas_dataframe()

In [ ]:
df

In [ ]:
df['normalized-mean'] = neutron_source * df['mean']

In [ ]:
# get all materials from the geometry
materials = pincell_model.geometry.get_all_materials()
# set names based on matching material IDs
for mat_id, material in materials.items():
    df.loc[df['material'] == mat_id, 'mat_name'] = material.name
df

In [ ]:
fission_df = df[df['score'] == 'fission']
fission_df

In [ ]:
fission_df.plot('mat_name', 'normalized-mean', kind='bar', ylabel='fissions / s')
_ = plt.xticks(rotation=30, ha='right')

In [ ]:
scatter_df = df[df['score'] == 'scatter']
scatter_df.plot('mat_name', 'mean', kind='bar', ylabel='scatters / s')
_ = plt.xticks(rotation=30, ha='right')

absorption_df = df[df['score'] == 'absorption']
absorption_df.plot('mat_name', 'mean', kind='bar', ylabel='absorptions / s')
_ = plt.xticks(rotation=30, ha='right')

<div class="alert alert-block alert-info">
Thank you for attending the workshop! <br/>
<b>For more examples, see: </b> <a href="https://github.com/openmc-dev/openmc-notebooks">https://github.com/openmc-dev/openmc-notebooks</a>
</div>